In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np
import torch
import os
from huggingface_hub import login
import wandb

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

print(raw_datasets)

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})


In [7]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.8, seed=20)

print(split_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 168138
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 42035
    })
})


In [8]:
split_datasets["validation"] = split_datasets.pop("test")
print(split_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 168138
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 42035
    })
})


In [9]:
def flatten_translation(examples):
    return {
        "en": [ex["en"] for ex in examples["translation"]],
        "fr": [ex["fr"] for ex in examples["translation"]]
    }

equivalent_datasets = split_datasets.map(flatten_translation, batched=True, remove_columns=["id", "translation"])

print(equivalent_datasets)

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 168138
    })
    validation: Dataset({
        features: ['en', 'fr'],
        num_rows: 42035
    })
})


In [10]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
en_sentence = equivalent_datasets["train"]["en"][0]
fr_sentence = equivalent_datasets["train"]["fr"][0]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
print(inputs)
print(tokenizer.decode(inputs["input_ids"]))
print(tokenizer.encode(en_sentence))
print(tokenizer.decode(inputs["labels"]))

{'input_ids': [1232, 13572, 7823, 9, 0], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [22181, 10691, 412, 9, 1232, 21332, 0]}
Web Shortcuts</s>
[1232, 13572, 7823, 9, 0]
Raccourcis WebComment</s>


In [17]:
max_length = 128
def preprocess_function(examples):
    inputs = examples["en"]
    targets = examples["fr"]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

# data_check = equivalent_datasets["train"][0:4]
# print(preprocess_function(data_check))
tokenized_datasets_eq = equivalent_datasets.map(preprocess_function, batched=True, remove_columns=equivalent_datasets["train"].column_names)

Map: 100%|██████████| 42035/42035 [00:11<00:00, 3803.06 examples/s]


In [12]:
max_length = 128
def preprocess_function2(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

# data_check = split_datasets["train"][0:10]
# print(preprocess_function2(data_check))
tokenized_datasets = split_datasets.map(preprocess_function2, batched=True, remove_columns=split_datasets["train"].column_names)

Map:   0%|          | 0/168138 [00:00<?, ? examples/s]

Map:   0%|          | 0/42035 [00:00<?, ? examples/s]

In [20]:
max_length = 128
def preprocess_function3(examples):
    inputs = examples["en"]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["fr"], max_length=max_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets_3 = equivalent_datasets.map(preprocess_function3, batched=True, remove_columns=equivalent_datasets["train"].column_names)

Map: 100%|██████████| 42035/42035 [00:10<00:00, 3924.09 examples/s]


In [13]:
print(tokenized_datasets["train"][0])
print(tokenized_datasets_eq["train"][0])
print(tokenized_datasets_3["train"][0])

{'input_ids': [1232, 13572, 7823, 9, 0], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [22181, 10691, 412, 9, 1232, 21332, 0]}


NameError: name 'tokenized_datasets_eq' is not defined

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [15]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1)])
print(batch.keys())
# print(batch["labels"])
# print(batch["input_ids"])
# print(batch["attention_mask"])

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])


In [16]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds): 
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != 100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [17]:
hf_login_key = os.environ.get("HF_LOGIN_KEY")
login(token=hf_login_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\kyanj\.cache\huggingface\token
Login successful


In [18]:
wandb_api_key = os.environ.get("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="translation_test"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="checkpoint"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kyanj\_netrc
wandb: Currently logged in as: kj821 (kj821-imperial-college-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [19]:
args = Seq2SeqTrainingArguments(
    f"Model_Files/translation-finetuning-test-en-fr",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
    report_to=["wandb"],
    run_name="translation-finetuning-test-en-fr-v1"
    )

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

  0%|          | 0/15765 [00:00<?, ?it/s]

{'loss': 1.5892, 'grad_norm': 3.893394947052002, 'learning_rate': 9.968284173802727e-05, 'epoch': 0.01}
{'loss': 1.482, 'grad_norm': 3.9422049522399902, 'learning_rate': 9.936568347605456e-05, 'epoch': 0.02}
{'loss': 1.3554, 'grad_norm': 4.29349422454834, 'learning_rate': 9.904852521408184e-05, 'epoch': 0.03}
{'loss': 1.3709, 'grad_norm': 4.0807085037231445, 'learning_rate': 9.873136695210911e-05, 'epoch': 0.04}
{'loss': 1.3018, 'grad_norm': 4.168679237365723, 'learning_rate': 9.841420869013638e-05, 'epoch': 0.05}
{'loss': 1.2962, 'grad_norm': 4.147677421569824, 'learning_rate': 9.809705042816366e-05, 'epoch': 0.06}
{'loss': 1.2682, 'grad_norm': 4.282947540283203, 'learning_rate': 9.777989216619093e-05, 'epoch': 0.07}
{'loss': 1.2609, 'grad_norm': 3.9418632984161377, 'learning_rate': 9.746273390421821e-05, 'epoch': 0.08}
{'loss': 1.2616, 'grad_norm': 3.8237521648406982, 'learning_rate': 9.714557564224549e-05, 'epoch': 0.09}
{'loss': 1.2836, 'grad_norm': 3.9868547916412354, 'learning_ra

  0%|          | 0/657 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
print(torch.cuda.is_available())

False


In [ ]:
from accelerate import Accelerator
from transformers import get_scheduler

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )